<a href="https://colab.research.google.com/github/Sartorio12/Projeto_Modulo_5/blob/main/notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Projeto Final do Módulo V</center>
<center> Um breve estudo sobre gastos parlamentares Brasileiros

<center><img src='https://i.imgur.com/QLJaHpa.png' style="width:200px">

<center><a href="https://colab.research.google.com/github/Sartorio12/Projeto_Modulo_5/blob/main/notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Grupo 5:** <a href="https://github.com/analumf" target="_blank">Analu Francisco</a> ; <a href="https://github.com/leticiafelippe" target="_blank">Letícia Felippe</a> ; <a href="https://github.com/sartorio12" target="_blank">Mateus Sartorio</a> ; <a href="www.linkedin.com/in/mateus-sartorio" target="blank">Mateus Sartório</a> ; <a href="https://github.com/hara0-tavares" target="_blank">Harão Tavares</a> ; <a href="https://github.com/jonathaslkc" target="_blank">Jonathas Carvalho</a>



<center><img src='https://i.imgur.com/QadqCNH.jpg' style="width:600px">
<center>Fonte: Agência Câmara de Notícias — Roque de Sá/Agência Senado

---

# **1.** Sobre o estudo e objetivos

O que você imagina quando ouve o conceito **"Cota Parlamentar"** ou **"Cota para o Exercício da Atividade Parlamentar (CEAP)"**?

Você sabia que é através da **CEAP** que deputados e senadores custeiam suas despesas em atividades parlamentares? O mais interessante disso, é que os dados da CEAP que iremos analisar hoje são abertos e podem ser acessados por qualquer pesso, ou seja, qualquer pessoa pode ver como os deputados e senadores utilizam essse tipo de recurso. 

Apesar de cobrir uma quantidade grande de despesas, os gastos com a CEAP devem seguir uma série de regras definidas pelo Ato da Mesa 43/2009. Isso quer dizer que os parlamentares não podem utilizar esses recursos de qualquer maneira. Alguns exemplos de gastos vedados são a aquisição de gêneros alimentícios (ex.: o parlamentar pode gastar com as suas refeições mas não pode comprar carne para o churrasco no fim de semana), gastos de caráter eleitoral e gastos com a participação em cursos de educação básica, graduação e pós-graduação.

Além disso, como citado anteriormente, a CEAP é destinada a custear gastos exclusivamente vinculados ao exercício da atividade parlamentar, o que veda o seu uso para o benefício de terceiros, como por exemplo, o pagamento de refeições para outras pessoas além do próprio parlamentar.
Apesar de todas essas regras, a fiscalização desses gastos é bastante falha, sendo comum o surgimento de denúncias sobre irregularidades.

Neste cenário surgiu um projeto chamado Operação Serenata de Amor ( https://serenata.ai/ ), que tem como objetivo utilizar ferramentas de ciência de dados para fiscalizar gastos públicos e compartilhar informações relevantes com o público. O projeto tem uma série de módulos. Alguns deles são responsáveis por baixar e limpar os dados, outros por identificar irregularidades e por fim existem alguns módulos responsáveis pela divulgação de eventuais alertas de irregularidades. Um desses módulos de divulgação é o bot @RosieDaSerenata, responsável por publicar alertas de irregularidades no Twitter.

<center><img src='https://i.imgur.com/AjyWlf1.jpg'>

**É sobre isso que vamos tratar nesta análise, que tem dois focos principais:**
- *Demonstrar como investigaremos esses dados utilizando Python.*
- *Elaborar conceitos a respeito da utilização da CEAP nos últimos 3 anos, com destaques no cenário de pandemia(COVID-19).*

#### <center> **Nós do grupo 5 esperamos que a leitura desse notebook seja esclarecedora e útil :)**

# **2.** Começando a nossa análise

Essa parte será um pouco teórica, pois nela exibiremos um pouco sobre a limpeza dos dados coletados no site da Câmara. Caso você queira somente acompanhar quais foram as nossas deduções baseadas nessas informações, pode pular para a parte X ( Preciso preencher a parte ).

*Antes de mais nada, vamos apresentar um glossário sobre o DataSet:*

- `Nome Parlamentar(txNomeParlamentar)` :  *Nome adotado pelo Parlamentar ao tomar posse do seu mandato.*
- `ideCadastro(Identificador Único do Parlamentar)` : *Número que identifica unicamente um deputado federal na CD.*
- `nuCarteiraParlamentar(Número da Carteira Parlamentar)` : *Documento usado para identificar um deputado federal na CD.*
- `nuLegislatura(Número da  Legislatura)` : *No contexto da cota CEAP, representa o ano base de início da legislatura.*
- `sgUF(Sigla da Unidade Federal)` : *Representa a UF do parlamentar.*
- `sdPartido(Sigla do Partido)` : *Representa o partido do parlamentar.*
- `codLegislatura(Código da Legislatura)` : *No contexto da cota CEAP, o seu conteúdo representa o código identificador da Legislatura.*
- `numSubCota(Número da Subcota) `: *Representa o código do Tipo de Despesa, comprovada por meio da emissão de um documento fiscal, a qual é debitada na cota do deputado.*
- `txtDescricao(Descrição da Subcota)` : *Descrição do Tipo de Despesa.*
- `numEspecificacaoSubCota(Número da Especificação da Subcota)` : *Representa o código desta especificação mais detalhada.*
- `txtDescricaoEspecificacao(Descrição da Especificação da Subcota)` : *Detalha o tipo de despesa efetuada.*
- `txtFornecedor(Fornecedor)` : *Representa o fornecedor do produto ou serviço presente no documento fiscal.*
- `txtCNPJCPF(CNPJ/CPF)` : *Indica se o fornecedor é CNPJ ou CPF , presente no documento fiscal.*
- `txtNumero(Número do Documento)` : *Indica o número do documento do emissor da NF em questão.*
- `indTipoDocumento(Indicativo de Tipo de Documento Fiscal)` : *Tipo de documento do fiscal – 0 para Nota Fiscal; 1 para Recibo; 2 para Despesa no Exterior.*
- `datEmissao(Data de Emissão)` : *A data de emissão do documento.*
- `vlrDocumento(Valor do Documento)` : *O seu conteúdo é o valor de face do documento fiscal ou o valor do documento que deu causa à despesa.*
- `vlrGlosa(Valor da Glosa)` : *Valor da glosa do documento que deu causa à despesa.*
- `vlrLiquido(Valor Liquido)` : *Valor líquido do documento (será calculado pela diferença entre o Valor do Documento e o Valor da Glosa)*
- `numMes(Mês) `: *Indica o mês da emissão da nota.*
- `numAno(Ano) `: *Indica o ano da emissão da nota.*
- `numParcela(Número da Parcela)` : *O seu conteúdo representa o número da parcela do documento fiscal. Ocorre quando o documento tem de ser reembolsado de forma parcelada.*
- `txtPassageiro(Nome do Passageiro)` : *O nome do passageiro, em caso da emissão do documento ser uma passagem aérea.*
- `txtTrecho(Trecho)` : *Representa qual o trecho da viagem, somente em casos de viagem aérea.*
- `numLote(Número do Lote) `: *Representa a capa do lote em que esse documento será incluído para ser entregue na Câmara para ressarcimento.*
- `numRessarcimento(Número do Ressarcimento) `: *Indica o ressarcimento do qual o documento faz parte.*
- `vlrRestituição(Valor da Restituição)` : *Indica o valor que será restituído sobre aquele documento.*
- `nuDeputadoId(Identificador do Solicitante)` : *Número que identifica um Parlamentar ou Liderança na Transparência da Cota para Exercício da Atividade Parlamentar.*

## **2.1.** Materiais e métodos

O presente trabalho foi desenvolvido utilizando `Google Colaboratory` e `Jupyter Notebook`, com a importação das seguintes bibliotecas:
> * `Pandas`: é uma biblioteca para uso em Python, open-source e de uso gratuito (sob uma licença BSD), que fornece ferramentas para análise e manipulação de dados. ;<br>
> * `Numpy`: é uma biblioteca para a linguagem Python com funções para se trabalhar com computação numérica. Seu principal objeto é o vetor n-dimensional, ou ndarray .;<br>
> * `Matplotlib`: é uma biblioteca com recursos para a geração de gráficos 2D a partir de arrays. Gráficos comuns podem ser criados com alta qualidade a partir de comandos simples, inspirados nos comandos gráficos do MATLAB;<br>
> * `Seaborn`: é uma biblioteca que atua em cima do matplotlib e ajuda a melhorar o visual dos gráficos, dando uma aparência mais bem acabada; e<br>
> * `Datetime`: é um módulo que fornece as classes para manipulação de datas e horas. Ainda que a aritmética de data e hora seja suportada, o foco da implementação é na extração eficiente do atributo para formatação da saída e manipulação.

A base de dados pode ser encontrada no site da Câmara (https://www.camara.leg.br/cota-parlamentar/), de acesso livre.

Para versionamento do código utilizamos o `Git` e `GitHub` para publicação do trabalho.

Para organização e acompanhamento das tarefas entre os integrantes do grupo foi utilizado o `Trello`.

As reuniões foram estabelecidas através da plataforma `Discord`.

## **2.2.** Importando nossas bibliotecas

A parte incial, simples importação de bibliotecas que serão utilizadas.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## **2.3.** Importando nossos dados

In [4]:
df1 = pd.read_csv('Ano-2019.csv', sep=';')
df2 = pd.read_csv('Ano-2020.csv', sep=';')
df3 = pd.read_csv('Ano-2021.csv', sep=';')

In [ ]:
# Criando uma cópia do dataframe como boa prática
df_copia = df_unico.copy()
df_copia.shape

In [ ]:
# Nome das colunas
df_copia.columns

In [ ]:
# Tipo de dado de cada coluna
df_copia.dtypes

In [ ]:
# Vendo todas as informações
df_copia.info()

observação: fizemos a consulta e achamos alguns valores nulos em algumas colunas mas achamos que elas não são relevantes pras consultas do nosso projeto.
è preciso analisar em grupo se devemos removê-las ou mantê-las.

In [ ]:
# Procurando numero de dados nulos em percentual
round(df_copia.isnull().mean()*100,2)

In [ ]:
# o describe foi feito pra destacar os valores negativos presentes nas colunas:'vlrDocumento' e 'vlrLiquido' 
df_copia.describe()

observação: Achamos importante explicar o que são esses valores negativos ( compensação de passagens aéreas) conforme dicionário do arquivo.
https://www2.camara.leg.br/transparencia/cota-para-exercicio-da-atividade-parlamentar/explicacoes-sobre-o-formato-dos-arquivos-xml

In [ ]:
# Visualizando as 5 primeiras linhas do dataframe
df_copia.head()

In [ ]:
# Visualizando as cinco últimas linhas do dataframe
df_copia.tail()

In [ ]:
# Buscamos amostra aleatória de cinco linhas
df_copia.sample(5)

In [ ]:
# Essa consulta fizemos um novo df, filtrando na coluna 'txtDescricao' valores diferentes de 'PASSAGEM AÉREA - RPA'  e 'PASSAGEM AÉREA - SIGEPA' e a partir desse novo df fazer os cálculos de despesas.
valores = df_copia[(df_copia['txtDescricao'] != 'PASSAGEM AÉREA - RPA')&(df_copia['txtDescricao'] != 'PASSAGEM AÉREA - SIGEPA')]
valores['txtDescricao'].unique()
valores.vlrLiquido

In [ ]:
# os dois valores negativos da coluna 'VrlLiquido'
valores.query('vlrLiquido < 0')['vlrLiquido']

In [ ]:
# Convertemos a coluna datEmissao de object para datatime e colocamos só pra mostrar o formato dia/mes/ano pq o horário é irrelevante para as consultas
df_copia['datEmissao'] = pd.to_datetime(df_copia['datEmissao'], format = '%Y/%m/%d')
df_copia['datEmissao']

In [ ]:
#Conferindo se o type da data foi transformado
df_copia.dtypes

In [ ]:
df_copia.shape

In [ ]:
# df_copia[df_copia['cpf'].isnull()]
df_copia.isnull().sum()


In [ ]:
df_copia['txNomeParlamentar'].str.contains('LIDMIN', na=False)

In [ ]:
sem_partido_sem_uf = valores[valores['sgUF'].isnull()][['txNomeParlamentar', 'sgUF', 'sgPartido','vlrLiquido']]

sem_partido_sem_uf

In [ ]:
sem_partido_sem_uf['txNomeParlamentar'].value_counts()

In [ ]:
df_unico.sgUF.value_counts()

In [ ]:
#Atribuindo os paritdo correspondentes as despesas atribuídas apenas a lideranças
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO PSL", 'PSL', df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO PT", 'PT', df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO PTB", 'PTB', df_copia.sgPartido) 
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO PSDB", 'PSDB', df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO PODEMOS", 'PODEMOS', df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO SOLIDARIEDADE", 'SOLIDARIEDADE', df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO PSD", 'PSD',df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO NOVO", 'NOVO',df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO DEMOCRATAS", 'DEMOCRATAS', df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO CIDADANIA", 'CIDADANIA', df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO AVANTE", 'AVANTE', df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO PDT", 'PDT', df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO PSOL", 'PSOL', df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO PROGRESSISTAS", 'PROGRESSISTAS',df_copia.sgPartido)
df_copia['sgPartido'] = np.where(df_copia['txNomeParlamentar'] == "LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL", 'SOCIAL', df_copia.sgPartido)

In [ ]:
#Verificando o funcionamento da atribuição
df_copia[df_copia['txNomeParlamentar'].str.match('LIDERANÇA')].sample(5)

### Análise dos DataFrames ( Harão e Jonathas)